In [1]:
using Memoize
using DataStructures

file = "input.txt"

inp = [split(x, "contains") for x in readlines(file)]
p = r"(?: a )?((?:.*)(?: generator|-compatible microchip))\.?"
types = Set()
things = [(1, "elevator")]
for (ind, line) in enumerate(inp)
    for val in split(replace(line[2], ", and" => ',', "and" => ','), ',')
        if occursin("nothing relevant", val)
            continue
        end
        m = match(p, val)
        push!(types, m[1])
        push!(things, (ind, m[1]))
    end
end
indexies = Dict(sort(collect(types)) .=> 2:length(types)+1)
indexies["elevator"] = 1
indexies

Dict{SubString{String}, Int64} with 11 entries:
  "cobalt-compatible microchip"     => 3
  "ruthenium generator"             => 10
  "cobalt generator"                => 2
  "ruthenium-compatible microchip"  => 11
  "plutonium generator"             => 6
  "promethium-compatible microchip" => 9
  "curium-compatible microchip"     => 5
  "elevator"                        => 1
  "plutonium-compatible microchip"  => 7
  "promethium generator"            => 8
  "curium generator"                => 4

In [2]:
things

11-element Vector{Tuple{Int64, String}}:
 (1, "elevator")
 (1, "promethium generator")
 (1, "promethium-compatible microchip")
 (2, "cobalt generator")
 (2, "curium generator")
 (2, "ruthenium generator")
 (2, "plutonium generator")
 (3, "cobalt-compatible microchip")
 (3, "curium-compatible microchip")
 (3, "ruthenium-compatible microchip")
 (3, "plutonium-compatible microchip")

In [3]:
grid = falses((length(inp), length(indexies)))
for thing in things
    grid[thing[1], indexies[thing[2]]] = 1
end
grid

4×11 BitMatrix:
 1  0  0  0  0  0  0  1  1  0  0
 0  1  0  1  0  1  0  0  0  1  0
 0  0  1  0  1  0  1  0  0  0  1
 0  0  0  0  0  0  0  0  0  0  0

In [4]:
function is_valid(state)
    # %2 == 1 is chip, %2==0 is generator
    for floor in eachrow(state)
        generators = floor[2:2:end]
        chips = floor[3:2:end]
        if any(chips-generators .> 0) & any(generators)
            return false
        end
    end
    return true
end

function is_done(state)
    return sum(state[1:size(state)[1]-1, :]) == 0 
    # nothing on floors outside the last one
end

is_valid(grid)

true

In [ ]:
struct Node
    grid
    cost
    h_cost
    elev
    function Node(grid, cost)
        grid = grid
        cost = cost
        h_cost = heuristic()
        elev = findfirst(grid[:,1])
        return new(grid, cost, h_cost, elev)
    end

    function heuristic()
        items = sum(grid[end:-1:1, 2:end], dims=2)
        out = sum(2 * sum(items[1:floor]) - 3 for floor in 1:size(grid)[1])
        return out
    end
end

Base.:<(x::Node, y::Node) = (x.cost+x.h_cost) < (y.cost+y.h_cost)

Base.:(==)(x::Node, y::Node) = (x.cost + x.h_cost) === (y.cost+y.h_cost)

function node_hasheq(x, y) #for hashing
    return x.hash == y.hash
end

function _popfirst!(tree)
    out = tree[1]
end

function A_star(starting_grid)
    n_floors = size(starting_grid)[1]
    moves = SplayTree{Node}()
    push!(moves, Node(starting_grid, 0))
    global_min = Inf
    while moves.count > 0
        _nxt = moves[1]
        delete!(moves, _nxt)
        # If the best case is worse than what we've found, ignore this path
        if (_nxt.h_cost + _nxt.cost) >= global_min
            continue
        end
        
        if is_done(_nxt.grid)
            global_min = _nxt.cost
            continue
        end
        _moves_taken = _nxt.cost
        elev = _nxt.elev
        occ = findall(_nxt.grid[elev, 2:end]) .+ 1 #occupied on floor
        
        #possible moves where things are moved up
        if elev < n_floors
            for first in 1:length(occ)
                #second item to be moved
                for sec in first+1:length(occ)
                    _state = copy(_nxt.grid)
                    _state[elev, 1] = 0
                    _state[elev, occ[first]] = 0
                    _state[elev, occ[sec]] = 0
                    _state[elev+1, 1] = 1
                    _state[elev+1, occ[first]] = 1
                    _state[elev+1, occ[sec]] = 1
                    push!(moves, Node(_state, _moves_taken+1))
                end
                #only move first 
                _state = copy(_nxt.grid)
                _state[elev, 1] = 0
                _state[elev, occ[first]] = 0
                _state[elev+1, 1] = 1
                _state[elev+1, occ[first]] = 1
                push!(moves, Node(_state, _moves_taken+1))
            end
        end

        #possible moves where things are moved down
        if elev > 1
            for first in 1:length(occ)
                #second item to be moved
                for sec in first+1:length(occ)
                    _state = copy(_nxt.grid)
                    _state[elev, 1] = 0
                    _state[elev, first] = 0
                    _state[elev, sec] = 0
                    _state[elev-1, 1] = 1
                    _state[elev-1, first] = 1
                    _state[elev-1, sec] = 1
                    push!(moves, Node(_state, _moves_taken+1))
                end
                #only move first 
                _state = copy(_nxt.grid)
                _state[elev, 1] = 0
                _state[elev, first] = 0
                _state[elev-1, 1] = 1
                _state[elev-1, first] = 1
                push!(moves, Node(_state, _moves_taken+1))
            end
        end
        
    end
    return global_min
end
A_star(grid)

moves = [(copy(grid), 0)]
seen = Set()
n_floors = size(grid)[1]
state = popfirst!(moves)[1]
_moves_taken = 0
__mvs = 0
_states_checked = 0
done_this_mv = false
min_this_mv = Inf
while !is_done(state)
    
    if _moves_taken > __mvs
        display("Calculates $(_states_checked) states at $(__mvs) moves taken")
        __mvs = _moves_taken
        _states_checked = 0
        if done_this_mv
            _moves_taken = min_this_mv
            break
        end
    end
    hashed_state, isdiag = grid_hash(state)
    if hashed_state in seen #prune seen states
        _next = popfirst!(moves)
        state = _next[1]
        _moves_taken = _next[2]
        continue
    end
    #if not seen, add to seen
    push!(seen, hashed_state)
    _states_checked += 1
    elev = findfirst(state[:,1]) #elevator's floor
    occ = findall(state[elev, 2:end]) .+ 1 #occupied on floor
    if isdiag #once all sets are paired, the equation per set is
        display(state)
        display("All paired, $(_moves_taken) moves in")
        if sum(state[elev+1:end, 2:end]) < 1 #make sure nothing is below the elevator
            items = sum(state[end:-1:1, 2:end], dims=2)
            display("items: $items")
            out = sum(2 * sum(items[1:floor]) - 3 for floor in 1:n_floors)
            done_this_mv = true
            display("Moves to finish: $out")
            __min = out + _moves_taken
            if __min < min_this_mv
                min_this_mv = __min
            end
        end
    end

    #possible moves where things are moved up
    if elev < n_floors
        for first in 1:length(occ)
            for sec in first+1:length(occ)
                _state = copy(state)
                _state[elev, 1] = 0
                _state[elev, occ[first]] = 0
                _state[elev, occ[sec]] = 0
                _state[elev+1, 1] = 1
                _state[elev+1, occ[first]] = 1
                _state[elev+1, occ[sec]] = 1
                push!(moves, (_state, _moves_taken+1))
            end
            #only move first 
            _state = copy(state)
            _state[elev, 1] = 0
            _state[elev, occ[first]] = 0
            _state[elev+1, 1] = 1
            _state[elev+1, occ[first]] = 1
            push!(moves, (_state, _moves_taken+1))
        end
    end
    
    #possible moves where things are moved down
    if elev > 1
        for first in 1:length(occ)
            for sec in first+1:length(occ)
                _state = copy(state)
                _state[elev, 1] = 0
                _state[elev, first] = 0
                _state[elev, sec] = 0
                _state[elev-1, 1] = 1
                _state[elev-1, first] = 1
                _state[elev-1, sec] = 1
                push!(moves, (_state, _moves_taken+1))
            end
            #only move first 
            _state = copy(state)
            _state[elev, 1] = 0
            _state[elev, first] = 0
            _state[elev-1, 1] = 1
            _state[elev-1, first] = 1
            push!(moves, (_state, _moves_taken+1))
        end
    end
    
    _next = popfirst!(moves)
    state = _next[1]
    _moves_taken = _next[2]
end
println("Part 1: $(_moves_taken)")

In [20]:
tree = SplayTree()
push!(tree, 25001)
tree[1]

25001

In [3]:
tree = SplayTree()


SplayTree{Any}(nothing, 0)

In [4]:
push!(tree, 100)
push!(tree, 200)
tree

SplayTree{Any}(DataStructures.SplayTreeNode{Any}(DataStructures.SplayTreeNode{Any}(nothing, nothing, DataStructures.SplayTreeNode{Any}(#= circular reference @-2 =#), 100), nothing, nothing, 200), 2)

In [18]:
for i=1:1000
    push!(tree, rand())
end
tree[1]

0.00011654109761227716

In [23]:
tree2 = SplayTree()
push!(tree2, 12)
delete!(tree2, 12)
tree2.count

0